# Notion Agent

In [1]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor notion-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 21.0 MB/s eta 0:00:00


In [2]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [3]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

In [ ]:
NOTION_TOKEN = "key here"
DATABASE_ID = "20f4296aca078044a20fd88ec8246774"
NOTION_ENDPOINT = 'https://api.notion.com/v1'
NOTION_PAGE_ID='20f4296aca078079949feb0f4dfa1d48'

In [5]:
#5th prompt
delegate_agent_prompt = """
You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent.

## TASK FLOW

1. If the user says "Create a new page named <title>", call `notion_page_agent(title=...)`.

2. If the user also wants to "write about ..." in the same request:
    - First, call `notion_page_agent(title=...)`.
    - The `notion_page_agent` internally uses a tool called `extract_page_id_from_notion_response` to extract the `page_id` from the response.
    - Once `page_id` is returned by `notion_page_agent`, call `notion_block_agent(page_id=..., content=...)`.

## PAGE_ID EXTRACTION

- The `notion_page_agent` tool will return the `page_id` directly using `extract_page_id_from_notion_response`.
- You do not need to parse the URL manually anymore. Use the `page_id` value returned from `notion_page_agent`.

## EXAMPLE:

If the user says:
"Create a new page named Ichigo. write about anime bleach in my notion page."

You must:

1. Call `notion_page_agent(title='Ichigo')`
2. Receive the response: `page_id = "2104296aca0781f08ef8caf4bf9ad809"`
3. Then call: `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content='write about anime bleach')`

---

If `notion_page_agent` fails to return a `page_id`, ask the user to specify or confirm the page manually.
"""


In [6]:
page_agent_prompt = """
You are a Notion agent responsible for managing pages within a Notion workspace. You're equipped with different tools.

Your primary tasks include:
1. **Creating Pages**: Create new pages with specified titles, content, and properties. Ensure pages are formatted according to user requests.

When performing actions:
- Always ensure page titles and content are clear and concise.
- If tool is not available to perform the action, simply say “I am unable to perform the request"
"""

In [7]:
block_agent_prompt = """
You are a Notion agent responsible for add blocks within Notion pages. Your tasks include:
    - **Creating Blocks**: Add new blocks to a page. Ensure the blocks are created with the proper content and formatting as specified by the user.

2. FORMATTING REQUIREMENTS:
  - Follow Notion's native styling
  - Maintain consistent spacing
  - Preserve hierarchical structure
  - Apply appropriate indentation

## ERROR HANDLING:
  - If a requested block type is unsupported: Respond with ‘Block type not supported’
  - If missing required parameters: Respond with ‘Missing required information: [parameter]’
  - For unauthorized actions: Respond with ‘Permission denied’
  - If tool is not available to perform the action: Respond with ‘I am unable to perform the request’

## Important:
  - When using add_notion_number_list_block, add_notion_bulleted_list_block, and add_notion_to_do_block functions, argument must be a
      list of str. For example
      add_notion_number_list_block([‘item 1°, ‘item 2‘, ‘item 3'])
      add_notion_bulleted_list_block([‘item 1', ‘item 2', ‘item 3'])
      add_notion_to_do_block([‘item 1', ‘item 2', ‘item 3'])
  - When passing a string containing a list of item, Use comma (",") as separator.
    For example, “Chapter 1, Chapter 2, "Chapter 3" is acceptable. “Chapter 1\nChapter 2\n Chapter 3" is not.
"""

In [ ]:
import os
from typing import List
import httpx
from langchain_core.tools import tool

NOTION_API_KEY = "key here"
NOTION_ENDPOINT = 'https://api.notion.com/v1'
NOTION_PAGE_ID='20f4296aca078079949feb0f4dfa1d48'

@tool
def create_notion_page(page_title):
  """
  Create a new page in Notion with the specified title and content.

  args:
      page_title: The title of the new page.
  """
  api_key = NOTION_API_KEY
  page_id = NOTION_PAGE_ID
  url = f'{NOTION_ENDPOINT}/pages'
  headers = {
      'Authorization': f'Bearer {api_key}',
      'Content-Type': 'application/json',
      'Notion-Version': '2022-06-28'
  }
  properties= {
        "title": [
            {
                "text": {
                    "content": page_title
                }
            }
        ]
    }
  try:
    response = httpx.post(url, headers=headers, json={
        'parent': {
            'page_id': page_id
        },
        'properties': properties
    })
    return response.json()
  except httpx.ReadTimeout:
    return {'error': 'Request timed out'}


In [9]:
@tool
def add_notion_heading_block(
        heading_type: str = 'heading_1',
        content: str = 'New Heading',
        is_hyperlink: bool = False,
        hyperlink_url: str = '',
        color: str = 'default'
):
  """
  Add a heading block to a Notion page with the specified content.

  args:
      context_variables: The context variables containing the API key and page ID.
      heading_type: The type of heading block to add (e.g., "heading_1', 'heading_2").
      content: The content of the heading block.
      is_hyperlink: A boolean value indicating whether the heading should be a hyperlink.
      hyperlink_url: The URL of the hyperlink.
      color: The color of the heading block.

  example usage:
      add_notion_heading_block (heading_type='heading_1', content='Lacinato Kale', is_hyperlink=False)
      add_notion_heading_block (heading_type='heading_1', content='Lacinato Kale', is_hyperlink=True, hyperlink_url='https://en.wikipedia.org/wiki/Lacinato_kale')
  """
  api_key = NOTION_API_KEY
  page_id = NOTION_PAGE_ID
  url = f'{NOTION_ENDPOINT}/blocks/{page_id}/children'
  headers = {
      'Authorization': f'Bearer {api_key}',
      'Content-Type': 'application/json',
      'Notion-Version': '2022-06-28'
  }
  text_obj = {'type': 'text', 'text': {'content': content}}
  data = {
      'children': [{
          'object': 'block',
          'type': heading_type,
          heading_type: {
              'rich_text': [text_obj],
              'color': color # Move color inside the heading block
          }
      }]
  }

  if is_hyperlink:
    text_obj['text']['link'] = {'url': hyperlink_url}

  try:
    response = httpx.patch(url, headers=headers, json=data)
    return response.json()
  except httpx.ReadTimeout:
    return {'error': 'Request timed out'}

In [10]:
@tool
def add_notion_paragraph_block(
    page_id: str,
    content: str,
    is_hyperlink: bool = False,
    hyperlink_url: str = '',
    color: str = 'default'
):
    """
    Add a paragraph block to a Notion page with the specified content.

    Args:
        page_id: The ID of the Notion page where the block will be added.
        content: The content of the paragraph block.
        is_hyperlink: Whether the paragraph is a hyperlink.
        hyperlink_url: The URL if it’s a hyperlink.
        color: The color of the paragraph block.

    Example usage:
        add_notion_paragraph_block(page_id='abc123', content='Hello', is_hyperlink=False)
        add_notion_paragraph_block(page_id='abc123', content='Click me', is_hyperlink=True, hyperlink_url='https://example.com')
    """
    url = f'{NOTION_ENDPOINT}/blocks/{page_id}/children'
    headers = {
        'Authorization': f'Bearer {NOTION_API_KEY}',
        'Content-Type': 'application/json',
        'Notion-Version': '2022-06-28'
    }

    text_obj = {'type': 'text', 'text': {'content': content}}
    if is_hyperlink:
        text_obj['text']['link'] = {'url': hyperlink_url}

    data = {
        'children': [{
            'object': 'block',
            'type': 'paragraph',
            'paragraph': {
                'rich_text': [text_obj],
                'color': color
            }
        }]
    }

    try:
        response = httpx.patch(url, headers=headers, json=data)
        return response.json()
    except httpx.ReadTimeout:
        return {'error': 'Request timed out'}


In [42]:
text = """To do breathing exercises, you can follow these steps:

1. **Proper Breathing**: Breathe in slowly and quietly, ensuring that a tiny thread placed in front of your nose does not move. Breathe out even more slowly than you inhaled. If possible, leave a short pause between your exhalation and the next inhalation. You can hold your breath for a few seconds if it's comfortable.

2. **One-Four-Two Breathing**: Inhale while repeating the Name of God or a mantra once. Hold your breath and repeat the same name four times. Exhale while repeating the name or mantra twice.

3. **Gradual Increase**: As you become more comfortable, you can gradually increase your counts, such as inhaling for four counts, holding for sixteen, and exhaling for eight.

4. **Alternate Nostril Breathing**: Press your right nostril with your thumb and inhale through the left nostril while repeating God's Name once. Hold your breath for four counts, repeating God's Name four times. Then, release your right nostril, press your left nostril, and exhale while repeating the name twice. Repeat this process starting with the left nostril.

These exercises are simple and effective, and they should not harm your lungs.
"""
add_notion_paragraph_block.run({
    "page_id": "21b4296aca078188a6a4d0d51d1f84f8",
    "content": text
})

{'object': 'list',
 'results': [{'object': 'block',
   'id': '21b4296a-ca07-813d-a48e-c038677aa616',
   'parent': {'type': 'page_id',
    'page_id': '21b4296a-ca07-8188-a6a4-d0d51d1f84f8'},
   'created_time': '2025-06-23T07:36:00.000Z',
   'last_edited_time': '2025-06-23T07:36:00.000Z',
   'created_by': {'object': 'user',
    'id': '774be3bb-5f8b-4662-a478-0dbd3fe7c835'},
   'last_edited_by': {'object': 'user',
    'id': '774be3bb-5f8b-4662-a478-0dbd3fe7c835'},
   'has_children': False,
   'archived': False,
   'in_trash': False,
   'type': 'paragraph',
   'paragraph': {'rich_text': [{'type': 'text',
      'text': {'content': "To do breathing exercises, you can follow these steps:\n\n1. **Proper Breathing**: Breathe in slowly and quietly, ensuring that a tiny thread placed in front of your nose does not move. Breathe out even more slowly than you inhaled. If possible, leave a short pause between your exhalation and the next inhalation. You can hold your breath for a few seconds if it's

In [11]:
from notion_client import Client

import re
from langchain.tools import tool


def extract_page_id_from_notion_response(response: str) -> str:
    """
    Extracts and returns the page id from a Notion URL within the given text.

    The function searches for a pattern matching a Notion URL that looks like:
      https://www.notion.so/<PageTitle>-<page_id>

    For example, given a response like:
      "I have created a new page titled **Orihime** in your Notion workspace. Additionally, I have created another page with the title **Orihime Inoue Description**.

       You can find the details in the pages linked below:

       - [Orihime](https://www.notion.so/Orihime-2104296aca0781e299b5d1e16f714891)"

    This function will return the page id:
      "2104296aca0781e299b5d1e16f714891"

    If no matching page id is found, it returns an explanatory message.
    """
    # This regex looks for a URL starting with https://www.notion.so/
    # followed by any characters (representing the title) and a hyphen,
    # then a 32-character hexadecimal string (the page id).
    pattern = r"https:\/\/www\.notion\.so\/[^-\s]+-([a-f0-9]{32})"
    match = re.search(pattern, response)
    if match:
        return match.group(1)
    else:
        return "No page id found in the response."


In [35]:
notion_page_agent = create_react_agent(
    model=model,
    tools=[create_notion_page, extract_page_id_from_notion_response],
    prompt=page_agent_prompt,
    name="notion_page_agent"
)

In [36]:
notion_block_agent = create_react_agent(
    model=model,
    tools=[add_notion_heading_block, add_notion_paragraph_block, extract_page_id_from_notion_response],
    prompt=block_agent_prompt,
    name="notion_block_agent"
)

In [44]:
from langchain_core.messages import HumanMessage

# Your content to add to Notion
text = """To do breathing exercises, you can follow these steps:

1. **Proper Breathing**: Breathe in slowly and quietly, ensuring that a tiny thread placed in front of your nose does not move. Breathe out even more slowly than you inhaled."""

# Example Notion response containing page URL
notion_response = """in your Notion workspace. Here is the link to the page: [Mindfulness111](https://www.notion.so/Mindfulness10111-21b4296aca07816bbe88dc310f6a2558?source=copy_link)."""

# Extract page ID from the response
try:
    page_id = extract_page_id_from_notion_response(notion_response)
    print(f"Extracted page ID: {page_id}")

    # Prepare the agent input
    input_data = {
        "messages": [
            HumanMessage(
                content=f"Add this content to page {page_id}: {text}"
            )
        ]
    }

    # Execute the agent
    response = notion_block_agent.invoke(input_data)
    print("Success:", response)

except Exception as e:
    print("Error:", str(e))

Extracted page ID: 21b4296aca07816bbe88dc310f6a2558
Success: {'messages': [HumanMessage(content='Add this content to page 21b4296aca07816bbe88dc310f6a2558: To do breathing exercises, you can follow these steps:\n\n1. **Proper Breathing**: Breathe in slowly and quietly, ensuring that a tiny thread placed in front of your nose does not move. Breathe out even more slowly than you inhaled.', additional_kwargs={}, response_metadata={}, id='c4caa0f6-7709-4830-9268-161d180ba519'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UHjWH3ozZ4XqaTGXcfi7rIhV', 'function': {'arguments': '{"page_id": "21b4296aca07816bbe88dc310f6a2558", "content": "To do breathing exercises, you can follow these steps:"}', 'name': 'add_notion_paragraph_block'}, 'type': 'function'}, {'id': 'call_1hy2q97T2TQxL2xrEWM1kl6h', 'function': {'arguments': '{"heading_type": "heading_2", "content": "1. Proper Breathing", "color": "default"}', 'name': 'add_notion_heading_block'}, 'type': 'function'}, {'id': '

# Meditation Agent

In [14]:
# Install required packages
!pip install -qU langchain==0.3.12 langchain-chroma langchain-community pypdf langchain-openai wikipedia langgraph

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
# Environment setup
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [16]:
# PDF Loading
from langchain.document_loaders import PyPDFLoader
pdf_files = ["27_Meditation_Techniques.pdf", "how-to-meditate.pdf"]
all_documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    docs = loader.load()
    all_documents.extend(docs)
documents = all_documents
print(f"Loaded {len(documents)} documents from {len(pdf_files)} PDFs.")

Loaded 39 documents from 2 PDFs.


In [17]:
# Text Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(f"Split into {len(texts)} chunks.")

Split into 89 chunks.


In [18]:
# Embedding
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [19]:
# Vectorstore (ChromaDB)
from langchain_chroma import Chroma as ch
persist_directory = "chroma_db"
db = ch.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [20]:
# Retriever with similarity score threshold
retriever = db.as_retriever(
    search_type="similarity", #change
    search_kwargs={"k": 4}
)

In [21]:
# RAG Prompt
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatOpenAI

custom_prompt_template = """
Use ONLY the pieces of information provided in the context to answer the user's question.
If the answer is not present in the context, explicitly return: I don't know.
Do NOT try to make up an answer or use outside knowledge.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(custom_prompt_template)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# RAG LCEL chain
rag_chain = (
    {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

/tmp/ipython-input-21-3785023118.py:16: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [22]:
from langchain.tools import tool
from typing import List
import requests
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chat_models import ChatOpenAI

# RAG Prompt Template
custom_prompt_template = """
Use ONLY the pieces of information provided in the context to answer the user's question.
If the answer is not present in the context, explicitly return: I don't know.
Do NOT try to make up an answer or use outside knowledge.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(custom_prompt_template)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# RAG Chain
rag_chain = (
    {
        "context": RunnableLambda(lambda x: retriever.invoke(x["question"])),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Define tool
@tool
def rag_tool(question: str) -> str:
    """
    Answers a question using a retrieval-augmented generation (RAG) approach.
    Retrieves relevant documents from a vector store and uses an LLM to generate the final answer.
    """
    return rag_chain.invoke({"question": question})


In [23]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

meditation_assistant = create_react_agent(
    model=model,
    tools=[rag_tool],
    prompt="You are a helpful Meditation assistant. Your job is to use RAG tool to Answers questions using the provided PDF documents. ",
    name="meditation_assistant"
)


In [24]:
from langchain_core.messages import (
    HumanMessage,
)
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="How to do breathing exercises?")
            ]
    }
    )
response["messages"]

[HumanMessage(content='How to do breathing exercises?', additional_kwargs={}, response_metadata={}, id='fa8c7e99-a810-4e74-8290-dfd17f51e004'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JVFrNRr8t9fbR5zwAVeUem72', 'function': {'arguments': '{"question":"How to do breathing exercises?"}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 102, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BlVY7oi3AyiAZ3eUk7YRftkEhcc2K', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--37309d02-edc6-4ec4-9bc6-83323a7b6ba3-0', tool_calls=[{'name': 'rag_tool'

In [25]:
response["messages"][-1].content

"To do breathing exercises, you can follow these steps:\n\n1. **Proper Breathing**: Breathe in slowly and quietly, ensuring that a tiny thread placed in front of your nose does not move. Breathe out even more slowly than you inhaled. If possible, leave a short pause between your exhalation and the next inhalation. You can hold your breath for a few seconds if it's comfortable.\n\n2. **One-Four-Two Breathing**: Inhale while repeating the Name of God or a mantra once. Hold your breath and repeat the same name four times. Exhale while repeating the name or mantra twice.\n\n3. **Gradual Increase**: As you become more comfortable, you can gradually increase your counts, such as inhaling for four counts, holding for sixteen, and exhaling for eight.\n\n4. **Alternate Nostril Breathing**: Press your right nostril with your thumb and inhale through the left nostril while repeating God's Name once. Hold your breath for four counts, repeating God's Name four times. Then, release your right nostri

In [26]:
text = response["messages"][-1].content

In [30]:
print(text)

To do breathing exercises, you can follow these steps:

1. **Proper Breathing**: Breathe in slowly and quietly, ensuring that a tiny thread placed in front of your nose does not move. Breathe out even more slowly than you inhaled. If possible, leave a short pause between your exhalation and the next inhalation. You can hold your breath for a few seconds if it's comfortable.

2. **One-Four-Two Breathing**: Inhale while repeating the Name of God or a mantra once. Hold your breath and repeat the same name four times. Exhale while repeating the name or mantra twice.

3. **Gradual Increase**: As you become more comfortable, you can gradually increase your counts, such as inhaling for four counts, holding for sixteen, and exhaling for eight.

4. **Alternate Nostril Breathing**: Press your right nostril with your thumb and inhale through the left nostril while repeating God's Name once. Hold your breath for four counts, repeating God's Name four times. Then, release your right nostril, press 

# Supervisor Agent

In [32]:
delegate_agent_prompt = """
You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate sub-agents: `notion_page_agent` and `notion_block_agent`.

---

## TASK FLOW

### 1. Creating a Page
- If the user says something like "Create a new page named <title>", call:
  `notion_page_agent(title=...)`

### 2. Creating a Page and Writing Content
- If the user also wants to "write about ..." in the same request:
  1. Call: `notion_page_agent(title=...)`
  2. Wait for the `page_id` from the response.
  3. Then call: `notion_block_agent(page_id=..., content=...)` where content is extracted from the user's message.

---

## HANDLING `text` VARIABLE

The supervisor may provide a variable `text` that contains the exact content to be written into a Notion page.

- If `text` is provided and a page title is also specified:
  1. Call: `notion_page_agent(title=...)`
  2. Receive `page_id` from the response
  3. Call: `notion_block_agent(page_id=..., content=text)`
  4. Do **not** rewrite, summarize, or generate new content — use `text` as-is.

- If `text` is provided but no page title is included:
  - Ask the user to specify the page title before proceeding.

- If `text` is **not** provided:
  - Use the original user message to infer or generate the content.

---

## PAGE_ID EXTRACTION

- The `notion_page_agent` tool automatically uses `extract_page_id_from_notion_response` to return the `page_id`.
- Do **not** extract or parse `page_id` from URLs yourself. Always rely on the returned value.

---

## EXAMPLES

### Example 1:
User: "Create a new page named Ichigo. Write about anime bleach."

- Call: `notion_page_agent(title='Ichigo')`
- Receive: `page_id = "2104296aca0781f08ef8caf4bf9ad809"`
- Call: `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content='write about anime bleach')`

### Example 2:
Given variable: `text = "Summary of my AI project: It's a Notion agent system using LangChain."`
User: "Create a new page named Project Summary."

- Call: `notion_page_agent(title='Project Summary')`
- Receive: `page_id = "a1b2c3d4..."`
- Call: `notion_block_agent(page_id='a1b2c3d4...', content=text)`

---

## FAILURE CASE

If `notion_page_agent` fails to return a `page_id`, respond to the user with a request to manually confirm or specify the page.

"""


In [ ]:
page_agent_prompt = """
You are a Notion agent responsible for managing pages in a Notion workspace. You are equipped with tools to create and manage pages.

## RESPONSIBILITIES

1. **Creating Pages**:
   - Create new pages with the specified title.
   - Optionally add initial content or metadata (properties) if provided.
   - Titles must be clear, concise, and match user instructions.

## BEHAVIOR RULES

- Respond clearly and concisely to user requests.
- If the required tool to perform an action is unavailable, respond with:
  **"I am unable to perform the request."**

## FORMATTING

- Apply Notion-native formatting standards where applicable.
- Do not add content unless explicitly requested or provided.
"""


In [34]:
block_agent_prompt = """
You are a Notion agent responsible for adding blocks to existing Notion pages.

## RESPONSIBILITIES

1. **Creating Blocks**:
   - Add new blocks to a given Notion page using the provided `page_id`.
   - Match the content type and format exactly as specified by the user.

## FORMATTING GUIDELINES

- Use Notion-native styles.
- Maintain consistent spacing and indentation.
- Preserve the correct hierarchical structure (e.g., headings, sub-bullets, etc.)

## ERROR HANDLING

Respond with the appropriate error message in these cases:
- Unsupported block type: **"Block type not supported"**
- Missing required parameters: **"Missing required information: [parameter]"**
- Unauthorized access: **"Permission denied"**
- Tool not available: **"I am unable to perform the request"**

## SPECIAL CASES: LIST BLOCKS

When using the following functions:
- `add_notion_number_list_block`
- `add_notion_bulleted_list_block`
- `add_notion_to_do_block`

**Argument Requirements**:
- Each must receive a list of strings.
  Example:
  ```python
  add_notion_number_list_block(["Item 1", "Item 2", "Item 3"])
"""

new

In [37]:
supervisor = create_supervisor(
    agents=[notion_page_agent, notion_block_agent  ],

    model=model,
    prompt=delegate_agent_prompt
).compile()



# Answers

In [59]:
extract_page_id_from_notion_response("""
    Extracts and returns the page id from a Notion URL within the given text.

    The function searches for a pattern matching a Notion URL that looks like:
      https://www.notion.so/<PageTitle>-<page_id>

    For example, given a response like:
      "I have created a new page titled **Orihime** in your Notion workspace. Additionally, I have created another page with the title **Orihime Inoue Description**.

       You can find the details in the pages linked below:

       - [Orihime](https://www.notion.so/Orihime-2104296aca0781e299b5d1e16f714891)""")

'2104296aca0781e299b5d1e16f714891'

In [ ]:
from langchain_core.messages import HumanMessage
import json
import re

from notion_client import Client

import re
from langchain.tools import tool


def extract_page_id_from_notion_response(response: str) -> str:
    """
    Extracts and returns the page id from a Notion URL within the given text.

    The function searches for a pattern matching a Notion URL that looks like:
      https://www.notion.so/<PageTitle>-<page_id>

    For example, given a response like:
      "I have created a new page titled **Orihime** in your Notion workspace. Additionally, I have created another page with the title **Orihime Inoue Description**.

       You can find the details in the pages linked below:

       - [Orihime](https://www.notion.so/Orihime-2104296aca0781e299b5d1e16f714891)"

    This function will return the page id:
      "2104296aca0781e299b5d1e16f714891"

    If no matching page id is found, it returns an explanatory message.
    """
    # This regex looks for a URL starting with https://www.notion.so/
    # followed by any characters (representing the title) and a hyphen,
    # then a 32-character hexadecimal string (the page id).
    pattern = r"https:\/\/www\.notion\.so\/[^-\s]+-([a-f0-9]{32})"
    match = re.search(pattern, response)
    if match:
        return match.group(1)
    else:
        return "No page id found in the response."



# ----------------------------
# STEP 1: Create the Notion page
# ----------------------------
page_title = "Mindfulness777"
create_page_msg = HumanMessage(
    content=f"Create a Notion page titled '{page_title}' and return the page URL so I can extract the page_id."
)

# Stream and collect the output
response_chunks = []
for chunk in supervisor.stream({"messages": [create_page_msg]}, {"recursion_limit": 100}):
    if  chunk:
        # response_chunks.append(chunk)
        print(chunk)

# Combine and extract page_id from the text response
# combined_response = " ".join(response_chunks)
# combined_response = response_chunks
# print(combined_response)

page_id = extract_page_id_from_notion_response(chunk)
print("Extracted page_id:", page_id)

response["messages"][-1].content

In [63]:
from langchain_core.messages import HumanMessage
import json
import re

from notion_client import Client

import re
from langchain.tools import tool


def extract_page_id_from_notion_response(response: str) -> str:
    """
    Extracts and returns the page id from a Notion URL within the given text.

    The function searches for a pattern matching a Notion URL that looks like:
      https://www.notion.so/<PageTitle>-<page_id>

    For example, given a response like:
      "I have created a new page titled **Orihime** in your Notion workspace. Additionally, I have created another page with the title **Orihime Inoue Description**.

       You can find the details in the pages linked below:

       - [Orihime](https://www.notion.so/Orihime-2104296aca0781e299b5d1e16f714891)"

    This function will return the page id:
      "2104296aca0781e299b5d1e16f714891"

    If no matching page id is found, it returns an explanatory message.
    """
    # This regex looks for a URL starting with https://www.notion.so/
    # followed by any characters (representing the title) and a hyphen,
    # then a 32-character hexadecimal string (the page id).
    pattern = r"https:\/\/www\.notion\.so\/[^-\s]+-([a-f0-9]{32})"
    match = re.search(pattern, response)
    if match:
        return match.group(1)
    else:
        return "No page id found in the response."



# ----------------------------
# STEP 1: Create the Notion page
# ----------------------------
page_title = "Mindfulness777"
create_page_msg = HumanMessage(
    content=f"Create a Notion page titled '{page_title}' and return the page URL so I can extract the page_id."
)

# Stream and collect the output
response_chunks = []
for chunk in supervisor.stream({"messages": [create_page_msg]}, {"recursion_limit": 100}):
    if  chunk:
        # response_chunks.append(chunk)
        print(chunk)

# Combine and extract page_id from the text response
# combined_response = " ".join(response_chunks)
# combined_response = response_chunks
# print(combined_response)

page_id = extract_page_id_from_notion_response(chunk)
print("Extracted page_id:", page_id)

# ----------------------------
# STEP 2: Add text block to that page
# ----------------------------
text = "Here are some detailed breathing techniques for practicing mindfulness meditation."
add_text_msg = HumanMessage(
    content=f"Add the following content to the Notion page with page_id `{page_id}`:\n\n{text}"
)

for chunk in supervisor.stream(
    {
        "messages": [add_text_msg],
        "text": json.dumps({
            "page_id": page_id,
            "content": text
        })
    },
    {"recursion_limit": 100},
):
    print(chunk)


{'supervisor': {'messages': [HumanMessage(content="Create a Notion page titled 'Mindfulness777' and return the page URL so I can extract the page_id.", additional_kwargs={}, response_metadata={}, id='b209b13a-667c-4ca8-85d3-6bf4f15de5fc'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BG9GKpuxrmKdVSx1xXDEy1pX', 'function': {'arguments': '{"title":"Mindfulness777"}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 722, 'total_tokens': 742, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BlXAOhI2DQiIepanqyPgLmRAbeEsG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervi

TypeError: expected string or bytes-like object, got 'AddableUpdatesDict'

In [54]:
from langchain_core.messages import HumanMessage
import json

# Content to add
text = "Here are some detailed breathing techniques for practicing mindfulness meditation."
page_title = "Mindfulness888"

# Step 1: Create the Notion page
create_page_instruction = HumanMessage(
    content=f"Create a Notion page titled '{page_title}' and return its page_id. Do not add any content yet."
)

# Step 2: Add text block to that page
add_text_instruction = HumanMessage(
    content=(
        f"Now take the page_id from the previous step and write the following text inside the page:\n\n"
        f"{text}"
    )
)

# Stream the messages in order
for chunk in supervisor.stream(
    {
        "messages": [create_page_instruction, add_text_instruction],
        "text": json.dumps({
            "page_title": page_title,
            "content": text
        })
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")


{'supervisor': {'messages': [HumanMessage(content="Create a Notion page titled 'Mindfulness888' and return its page_id. Do not add any content yet.", additional_kwargs={}, response_metadata={}, id='8838f437-4489-4624-af20-c59ad3b78c4d'), HumanMessage(content='Now take the page_id from the previous step and write the following text inside the page:\n\nHere are some detailed breathing techniques for practicing mindfulness meditation.', additional_kwargs={}, response_metadata={}, id='0ba7f954-e1d3-4895-ad44-6e1f2dbcf4cb'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7bDWpe5oQ6hSuT5NGQWwCiTC', 'function': {'arguments': '{"title":"Mindfulness888"}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 755, 'total_tokens': 775, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}

In [53]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import Runnable
import re

# ---------- STEP 1: Define the content ----------
content = {
    "page_title": "Mindfulness Techniques 222",
    "blocks": [
        {
            "type": "heading_2",
            "text": "Mindfulness Breathing Techniques"
        },
        {
            "type": "heading_3",
            "text": "1. Diaphragmatic Breathing"
        },
        {
            "type": "bulleted_list",
            "items": [
                "Sit comfortably with one hand on chest, one on belly",
                "Inhale deeply through nose (4 sec), feel belly rise",
                "Hold briefly (2 sec)",
                "Exhale slowly through mouth (6 sec)"
            ]
        },
        {
            "type": "heading_3",
            "text": "2. 4-7-8 Technique"
        },
        {
            "type": "bulleted_list",
            "items": [
                "Inhale quietly through nose (4 sec)",
                "Hold breath (7 sec)",
                "Exhale completely through mouth (8 sec)",
                "Repeat 4 cycles"
            ]
        },
        {
            "type": "heading_3",
            "text": "3. Box Breathing"
        },
        {
            "type": "bulleted_list",
            "items": [
                "Inhale (4 sec)",
                "Hold (4 sec)",
                "Exhale (4 sec)",
                "Hold (4 sec)",
                "Repeat"
            ]
        }
    ]
}
page_creation_message = HumanMessage(
    content=f"Create a Notion page with title: {content['page_title']}"
)

page_response = notion_page_agent.invoke({"messages": [page_creation_message]})

def extract_page_id_from_response(response_text):
    # Assumes Notion page URL is in the response and contains a 32-char hex ID
    match = re.search(r"[0-9a-f]{32}", str(response_text))
    return match.group(0) if match else None

page_id = extract_page_id_from_response(page_response)

if not page_id:
    raise ValueError("Page ID could not be extracted from response.")

# ---------- STEP 4: Add content blocks ----------
block_creation_message = HumanMessage(
    content=f"Add the following content blocks to the page with ID: {page_id}."
)

block_payload = {
    "page_id": page_id,
    "blocks": content["blocks"]
}

notion_block_agent.invoke({
    "messages": [block_creation_message],
    "text": str(block_payload)
})

print(f"✅ Page '{content['page_title']}' created and blocks added to Notion page: {page_id}")

✅ Page 'Mindfulness Techniques 222' created and blocks added to Notion page: 21b4296aca07814c887fff34af99c645


In [52]:
from langchain_core.messages import HumanMessage

# Define content with explicit block structure
content = {
    "page_title": "Mindfulness Techniques",
    "blocks": [
        {
            "type": "heading_2",
            "text": "Mindfulness Breathing Techniques"
        },
        {
            "type": "heading_3",
            "text": "1. Diaphragmatic Breathing"
        },
        {
            "type": "bulleted_list",
            "items": [
                "Sit comfortably with one hand on chest, one on belly",
                "Inhale deeply through nose (4 sec), feel belly rise",
                "Hold briefly (2 sec)",
                "Exhale slowly through mouth (6 sec)"
            ]
        },
        {
            "type": "heading_3",
            "text": "2. 4-7-8 Technique"
        },
        {
            "type": "bulleted_list",
            "items": [
                "Inhale quietly through nose (4 sec)",
                "Hold breath (7 sec)",
                "Exhale completely through mouth (8 sec)",
                "Repeat 4 cycles"
            ]
        },
        {
            "type": "heading_3",
            "text": "3. Box Breathing"
        },
        {
            "type": "bulleted_list",
            "items": [
                "Inhale (4 sec)",
                "Hold (4 sec)",
                "Exhale (4 sec)",
                "Hold (4 sec)",
                "Repeat"
            ]
        }
    ]
}

# Clear instruction with explicit steps
instruction = """Please perform these actions in sequence:
1. Create ONE new Notion page with title: {page_title}
2. Get the page_id from the creation response
3. Add all content blocks to that single page using the page_id
4. Do NOT create separate pages for each section

Content blocks to add:
{blocks}
""".format(
    page_title=content["page_title"],
    blocks="\n".join(
        f"- {block['type']}: {block['text'] if 'text' in block else ', '.join(block['items'])}"
        for block in content["blocks"]
    )
)

# Execute with supervisor
for chunk in supervisor.stream(
    {
        "messages": [HumanMessage(content=instruction)],
        "text": str(content)  # Pass structured data
    },
    {"recursion_limit": 200},
):
    if chunk:
        print(chunk)
    elif 'tool_calls' in chunk.get('additional_kwargs', {}):
        print(f"Action: {chunk['additional_kwargs']['tool_calls']}")
    print("---")

{'supervisor': {'messages': [HumanMessage(content='Please perform these actions in sequence:\n1. Create ONE new Notion page with title: Mindfulness Techniques\n2. Get the page_id from the creation response\n3. Add all content blocks to that single page using the page_id\n4. Do NOT create separate pages for each section\n\nContent blocks to add:\n- heading_2: Mindfulness Breathing Techniques\n- heading_3: 1. Diaphragmatic Breathing\n- bulleted_list: Sit comfortably with one hand on chest, one on belly, Inhale deeply through nose (4 sec), feel belly rise, Hold briefly (2 sec), Exhale slowly through mouth (6 sec)\n- heading_3: 2. 4-7-8 Technique\n- bulleted_list: Inhale quietly through nose (4 sec), Hold breath (7 sec), Exhale completely through mouth (8 sec), Repeat 4 cycles\n- heading_3: 3. Box Breathing\n- bulleted_list: Inhale (4 sec), Hold (4 sec), Exhale (4 sec), Hold (4 sec), Repeat\n', additional_kwargs={}, response_metadata={}, id='fe452797-cedd-4fb9-bd74-ff28360ab616'), AIMessag

In [50]:
from langchain_core.messages import HumanMessage

# Define content with proper Notion block structure
meditation_content = {
    "heading": "Mindfulness Breathing Techniques",
    "sections": [
        {
            "title": "1. Diaphragmatic Breathing",
            "content": """
            - Sit comfortably with one hand on chest, one on belly
            - Inhale deeply through nose (4 sec), feel belly rise
            - Hold briefly (2 sec)
            - Exhale slowly through mouth (6 sec)
            """
        },
        {
            "title": "2. 4-7-8 Technique",
            "content": """
            - Inhale quietly through nose (4 sec)
            - Hold breath (7 sec)
            - Exhale completely through mouth (8 sec)
            - Repeat 4 cycles
            """
        },
        {
            "title": "3. Box Breathing",
            "content": """
            - Inhale (4 sec)
            - Hold (4 sec)
            - Exhale (4 sec)
            - Hold (4 sec)
            - Repeat
            """
        }
    ]
}

# Structured instruction
instruction = """Please perform these actions:
1. Create a new Notion page with ONLY the title "Mindfulness988"
2. Then add the following structured content as separate blocks:
   - Heading: {heading}
   - Sections:
     {sections}
""".format(
    heading=meditation_content["heading"],
    sections="\n     ".join(
        f"{section['title']}\n     {section['content']}"
        for section in meditation_content["sections"]
    )
)

for chunk in supervisor.stream(
    {
        "messages": [HumanMessage(content=instruction)],
        "text": str(meditation_content)  # Pass structured data
    },
    {"recursion_limit": 200},  # Increased limit for complex operations
):
    if chunk:
        print(chunk )
    elif 'tool_calls' in chunk.get('additional_kwargs', {}):
        print(f"Action: {chunk['additional_kwargs']['tool_calls']}")
    print("---")

{'supervisor': {'messages': [HumanMessage(content='Please perform these actions:\n1. Create a new Notion page with ONLY the title "Mindfulness988"\n2. Then add the following structured content as separate blocks:\n   - Heading: Mindfulness Breathing Techniques\n   - Sections:\n     1. Diaphragmatic Breathing\n     \n            - Sit comfortably with one hand on chest, one on belly\n            - Inhale deeply through nose (4 sec), feel belly rise\n            - Hold briefly (2 sec)\n            - Exhale slowly through mouth (6 sec)\n            \n     2. 4-7-8 Technique\n     \n            - Inhale quietly through nose (4 sec)\n            - Hold breath (7 sec)\n            - Exhale completely through mouth (8 sec)\n            - Repeat 4 cycles\n            \n     3. Box Breathing\n     \n            - Inhale (4 sec)\n            - Hold (4 sec)\n            - Exhale (4 sec)\n            - Hold (4 sec)\n            - Repeat\n            \n', additional_kwargs={}, response_metadata={},

In [49]:
from langchain_core.messages import HumanMessage

# Define high-quality content with clear structure
meditation_content = """
1. Diaphragmatic Breathing
- Sit comfortably with one hand on chest, one on belly
- Inhale deeply through nose (4 sec), feel belly rise
- Hold briefly (2 sec)
- Exhale slowly through mouth (6 sec)

2. 4-7-8 Technique
- Inhale quietly through nose (4 sec)
- Hold breath (7 sec)
- Exhale completely through mouth (8 sec)
- Repeat 4 cycles

3. Box Breathing
- Inhale (4 sec)
- Hold (4 sec)
- Exhale (4 sec)
- Hold (4 sec)
- Repeat
"""

# Structured instruction with clear steps
instruction = """Please perform these actions sequentially:
1. Create a new Notion page titled "Mindfulness900"
2. Extract the page ID from the creation response
3. Add the following formatted content to that page:
{content}""".format(content=meditation_content)

for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content=instruction,
            )
        ],
        "text": meditation_content  # Pass the well-formatted content
    },
    {"recursion_limit": 150},
):
    # Print the execution trace
    if 'content' in chunk:
        print(chunk['content'])
    elif 'tool_calls' in chunk.get('additional_kwargs', {}):
        print(f"Tool call: {chunk['additional_kwargs']['tool_calls']}")
    print("---\n")

---

---

---



In [46]:
from langchain_core.messages import HumanMessage

# Define the text content you want to add to the Notion page
text = "Here are some detailed breathing techniques for practicing mindfulness meditation."

for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content=f"Create a meditation page called Mindfulness999, get its page id then write this {text} inside that page."
            )
        ],
        "text": text  # This will be used as the content for the page
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a meditation page called Mindfulness999, get its page id then write this Here are some detailed breathing techniques for practicing mindfulness meditation. inside that page.', additional_kwargs={}, response_metadata={}, id='b5088df4-df94-457d-b581-fd3dcdfce642'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9UDa0mQNEICCSaGe7kZGVDmj', 'function': {'arguments': '{"title":"Mindfulness999"}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 729, 'total_tokens': 749, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BlWAVkeoxU005MIzXrUafAPAQ7ngG', 'service

In [38]:
from langchain_core.messages import HumanMessage

# text = "Here are some detailed breathing techniques for practicing mindfulness meditation."

for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a meditation page called Mindfulness10111 and write about breathing techniques."
            )
        ],
        "text": text  # 👈 Add the `text` variable to the input payload
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")


{'supervisor': {'messages': [HumanMessage(content='Create a meditation page called Mindfulness10111 and write about breathing techniques.', additional_kwargs={}, response_metadata={}, id='29e946fe-0d9f-4894-83ab-6928bb309caa'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_D27bx0ggbnkLDhwwbJlQ1EF9', 'function': {'arguments': '{"title":"Mindfulness10111"}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 713, 'total_tokens': 734, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BlVjAAYZcmrSLvJMf8CsOSB2UluDD', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='

In [31]:
from langchain_core.messages import HumanMessage

text = """To do breathing exercises, you can follow these steps:

1. **Proper Breathing**: Breathe in slowly and quietly, ensuring that a tiny thread placed in front of your nose does not move. Breathe out even more slowly than you inhaled. If possible, leave a short pause between your exhalation and the next inhalation. You can hold your breath for a few seconds if it's comfortable.

2. **One-Four-Two Breathing**: Inhale while repeating the Name of God or a mantra once. Hold your breath and repeat the same name four times. Exhale while repeating the name or mantra twice.

3. **Gradual Increase**: As you become more comfortable, you can gradually increase your counts, such as inhaling for four counts, holding for sixteen, and exhaling for eight.

4. **Alternate Nostril Breathing**: Press your right nostril with your thumb and inhale through the left nostril while repeating God's Name once. Hold your breath for four counts, repeating God's Name four times. Then, release your right nostril, press your left nostril, and exhale while repeating the name twice. Repeat this process starting with the left nostril.

These exercises are simple and effective, and they should not harm your lungs.
"""

for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a meditation page called Mindfulness102 and write about breathing techniques."
            )
        ],
        "text": text  # 👈 THIS IS CRUCIAL: inject it here
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")


{'supervisor': {'messages': [HumanMessage(content='Create a meditation page called Mindfulness102 and write about breathing techniques.', additional_kwargs={}, response_metadata={}, id='0d73ba38-55bf-4eef-9ac5-b55468acff96'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IC0dAjODGQKXZBckuXgYSd2x', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 648, 'total_tokens': 662, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BlVeXsNNOjQQDD3uPiNSs6FuSVGk8', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--76e329c7-5b10-49a5-8d07

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a meditation page called Mindfulness101 and write about breathing techniques."

            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a meditation page called Mindfulness101 and write about breathing techniques.', additional_kwargs={}, response_metadata={}, id='537ca484-3ab0-466c-9525-e570bd54c7df'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3hmRJAP4PJ988iaULTif8CnA', 'function': {'arguments': '{"title":"Mindfulness101"}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 726, 'total_tokens': 746, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiLw03QqvdzEMzgpAqNiYD2FewPV6', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-

In [ ]:
delegate_agent_prompt = """
You are a Notion Supervisor Agent that strictly enforces these workflows:

1. FOR MEDITATION PAGE CREATION:
   - FIRST call `notion_page_agent(title="...")` and WAIT for page_id
   - THEN call `meditation_assistant(query="...")` and WAIT for content
   - FINALLY call `notion_block_agent(page_id=page_id, content=content)`

2. CONTENT VALIDATION:
   - Verify meditation content is non-empty before sending to block agent
   - If content is empty, retry meditation_assistant with refined query

3. ERROR HANDLING:
   - If block addition fails, RETRY with simplified content
   - Never proceed to next step without successful completion
   - Report EXACTLY where failures occur

4. OUTPUT FORMAT:
   - Confirm EACH step completion
   - Show preview of content being added
   - Provide final page URL
"""

In [ ]:
# Add this to your supervisor stream call
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a page called Lumine1 and write about meditation techniques"
            )
        ],
    },
    {
        "recursion_limit": 150,
        "debug": True  # Enable verbose logging
    },
):
    print(chunk)
    print("---")  # Add separator for clarity

{'supervisor': {'messages': [HumanMessage(content='Create a page called Lumine1 and write about meditation techniques', additional_kwargs={}, response_metadata={}, id='b4b49f7e-5c6d-46aa-8e5c-3a1e9dddbc2c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HQirCLcP6FnLwvPbKSjBHAlB', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 724, 'total_tokens': 738, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiFyUKM1gibk1fq3Aha78cujggJkl', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--3e908045-2e1b-46b2-b239-b640f9c2b994-0', 

In [ ]:
extract_page_id_from_notion_response("https://www.notion.so/Lumine1-2124296aca0781b78197c2f6dea347e9?source=copy_link")

'2124296aca0781b78197c2f6dea347e9'

In [ ]:
# Test 1: Page creation only
# Test Notion Page Agent
page_response = notion_page_agent.invoke({
    "messages": [HumanMessage(content="Create a page called TestPage")]
})

print(page_response)  # Check for page_id

# Test 2: Meditation content only
# meditation_content = meditation_assistant(query="meditation techniques")
meditation_content = meditation_assistant.invoke({
    "messages": [HumanMessage(content="meditation techniques")]
})
print(meditation_content)  # Should be plain text

# Test 3: Block addition
block_response = notion_block_agent.invoke({
    "messages": [HumanMessage(page_id=extract_page_id_from_notion_response("https://www.notion.so/Lumine1-2124296aca0781b78197c2f6dea347e9?source=copy_link"),
    content="Test content")]
})
# block_response = notion_block_agent(
#     page_id="2124296aca0781b78197c2f6dea347e9",  # Use a real ID
#     content="Test content"
# )
print(block_response)  # Check for success

{'messages': [HumanMessage(content='Create a page called TestPage', additional_kwargs={}, response_metadata={}, id='9fae003a-27ce-41c8-bdec-82ab8b2e1281'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UqKM1PnTwOjiHELljcrBFpkX', 'function': {'arguments': '{"page_title":"TestPage"}', 'name': 'create_notion_page'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 367, 'total_tokens': 385, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiGGRWE0CQBgqKjCBGSFilENhA5C5', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='notion_page_agent', id='run--84e21ed7-d1eb-4c34-ac7d-2d088a9f9f46-0', tool_calls=[{'name': 'create_noti

In [ ]:
meditation_content = meditation_assistant.invoke({
    "messages": [HumanMessage(content="explain breathing exerciese")]
})
print(meditation_content["messages"])

[HumanMessage(content='explain breathing exerciese', additional_kwargs={}, response_metadata={}, id='14a1841d-6691-4053-9433-c4cf591f6892'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kAD57o56GloaNw176J5zqZ0S', 'function': {'arguments': '{"question":"explain breathing exercises"}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 158, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiLu7Qo4AsU6ZM85ZS6bF3kD4Z32v', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--b4b1974b-7f0b-4651-aa78-7693abc4d1f2-0', tool_calls=[{'name': 'rag_tool', 'args

# Prompt